# PHYS 105A:  Introduction to Scientific Computing

## Random Numbers and Monte Carlo Methods

* In physical science, students very often start with the concept that everything can be done exactly and deterministically.

* This can be one of the biggest misconcept!

* Many physical processes are non-deterministic by nature.  Examples include:
  * Radioactive decay
  * Quantum mechanics

* Sometime, even when the governing equations are deterministic, the results are still non-deterministic.  Examples include:
  * 3-body problem: https://www.youtube.com/watch?v=D2YhKaANbWE&ab_channel=PhysicsSimulations
  * Brownian motion
  * Thermal noise

* Therefore, in computer we need some way to model these non-deterministic systems.

From Wikipedia:

Monte Carlo methods, or Monte Carlo experiments, are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle. They are often used in physical and mathematical problems and are most useful when it is difficult or impossible to use other approaches. Monte Carlo methods are mainly used in three problem classes: 
* optimization
* numerical integration
* generating draws from a probability distribution

Question:
    
* how do you find out magnetic field lines?
* how do you know your structure-finding algorithm is robust (for example, you make your own program to find all bubbles in an image of the Galaxy)?

In [ ]:
# Let's try python's random number library

import random as rnd

print(rnd.random())             # return a random float in the range [0,1)
print(rnd.randrange(100))       # return a random int in the range [0, 100)
print(rnd.randint(a=0,b=99))    # return a random int in the range [a, b+1)
print(rnd.gauss(mu=0, sigma=1)) # sample from a Gaussian distribution

In [ ]:
# We may plot the results of these random number generators

from matplotlib import pyplot as plt

Rs = [rnd.random() for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.randrange(100) for i in range(1000)]
plt.hist(Rs)

In [ ]:
Rs = [rnd.gauss(0, 1) for i in range(1000)]
plt.hist(Rs)

In [ ]:
# We may even show the random numbers in image

plt.imshow([[rnd.gauss(0,1) for i in range(100)] for j in range(100)])

In [ ]:
# There is also a seed() function

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])

rnd.seed(1234)
print([rnd.randrange(100) for i in range(10)])

# change the seed number and run again, do you see the difference?

* Once we have a random number generator, we are ready to develop Monte Carlo methods!

* We will start with a simple example of random walk.  The model is very simple:
  * We start with a (drunk) person at the center of the street.
  * As time t to t+1, the person randomly move one way +1 or the other -1 along the street.
  * The problem is, after n steps, how far away is the person away from the center of the street?

In [ ]:
# let's set up this problem in the following way:

T = range(1, 1000+1) # time step
X = [0] # initial position

for t in T:
    last = X[-1]            # last position of list X
    r    = rnd.randint(0,1) # we generate 0 or 1 randomly, see above for randint
    if r == 0:              # depending on r, we step +1 or -1
        curr = last + 1
    else:
        curr = last - 1
    X.append(curr)          # append the current position to the list X

# let's plot this random walk
plt.plot(X)

# can we convert these codes into a function? what would be the input and the output?

In [ ]:
# In order to find out how random walk behave statistically,
# we want to be able to run many simulations!

# It is convenient to define a function

def randomwalk(n_steps=1000):
    X = [0] # initial position
    
    for t in range(n_steps):
        last = X[-1]            # last position
        r    = rnd.randint(0,1) # we generate 0 or 1 randomly
        if r == 0:              # depending on r, we step left or right
            curr = last + 1
        else:
            curr = last - 1    
        X.append(curr)          # append the current position to the list X
        
    return X                    # return the result

# And we can use this function in another loop.
for i in range(10):
    plt.plot(randomwalk())

In [ ]:
# We may now ask how far away the peron would walk depending on the number of steps.

D = []
for t in T:
    X = randomwalk(t)
    D.append(abs(X[-1]))
    
plt.plot(D)

# Clearly, the distance gets farther when the number of steps increase.
# But this figure is too noisy to show the dependency.

In [ ]:
# There are multiple ways to make the above figure less noise.
# One way is to simply do multiple numerical experiments for the same number of steps.
# And obtain the average distance.

n_trials = 100
D = []
for t in T:
    M = 0
    for trial in range(n_trials):
        X  = randomwalk(t)
        M += abs(X[-1])
    M /= n_trials
    D.append(M)
    
plt.plot(D)

# The plot is much better!

In [ ]:
# let's plot this in log-log scale.
# And compare it with the law of diffusion D ~ sqrt(T)

plt.loglog(T, D)
plt.plot(T, [t**0.5 for t in T])

* You may use this simple random walk model to model real physical process.

* For example, the Brownian motion, which describe how pollen is randomly pushed by water molecules.

![Brownian motion](https://upload.wikimedia.org/wikipedia/commons/c/c2/Brownian_motion_large.gif)

* Einstein published a paper on Brownian motion in 1905, which is one of his first major scientific contributions.
* Einstein argued that the displacement of a Brownian particle is not proportional to the elapsed time, but rather to its square root.

![Einstein paper](https://books.google.com/books/content?id=6E90uQAACAAJ&printsec=frontcover&img=1&zoom=1&imgtk=AFLRE73LNhOSPC3TgN-nQbwWZcT9e6HMW3OBrA2xXzu0xA3a-yB5eLs6UFMVMObtICAS4stFGkBRnNIUShEtCNjR2WspNO0vs46rB07TYcmRBZn1WF0SGWziM5n39N757D7XX5-udBYg)

In [ ]:
# The simplest model of Brownian motion is a two-dimension random walk.

X = randomwalk()
Y = randomwalk()

plt.figure(figsize=(12,12))
plt.plot(X, Y)
plt.gca().set_aspect('equal')

# The resulting plot looks slightly funny because random walk forces x and y both to move at exactly one step.
# The final outcome is that the particle can only move in diagonal directions.

In [ ]:
# But this artifact becomes irrelevant when we model random walk for many many more steps.

X = randomwalk(100000)
Y = randomwalk(100000)

plt.figure(figsize=(12,12))
plt.plot(X, Y)
plt.gca().set_aspect('equal')